## Imports

In [377]:
import datetime
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold, chi2, RFE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, MinMaxScaler
from collections import Counter
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
import random
import xgboost


## Data Ingestion

In [378]:
df_train = pd.read_csv("af2_dataset_training_labeled.csv.gz", index_col=0)
df_train

,annotation_sequence,feat_A,feat_C,feat_D,feat_E,feat_F,feat_G,feat_H,feat_I,feat_K,...,feat_DSSP_10,feat_DSSP_11,feat_DSSP_12,feat_DSSP_13,coord_X,coord_Y,coord_Z,entry,entry_index,y_Ligand
0,M,False,False,False,False,False,False,False,False,False,...,0,0.0,47,-0.0,-26.499001,-4.742000,-35.189999,GEMI5_HUMAN,0,False
1,G,False,False,False,False,False,True,False,False,False,...,0,0.0,0,0.0,-25.158001,-1.342000,-34.104000,GEMI5_HUMAN,1,False
2,Q,False,False,False,False,False,False,False,False,False,...,1,-0.0,-1,-0.0,-21.926001,-1.641000,-32.175999,GEMI5_HUMAN,2,False
3,E,False,False,False,True,False,False,False,False,False,...,706,-0.1,705,-0.0,-22.073999,0.654000,-29.171000,GEMI5_HUMAN,3,False
4,P,False,False,False,False,False,False,False,False,False,...,0,0.0,705,-0.2,-19.783001,2.670000,-26.858999,GEMI5_HUMAN,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,S,False,False,False,False,False,False,False,False,False,...,-3,-0.1,2,-0.4,-19.742001,20.796000,-12.319000,AOC3_HUMAN,755,False
756,H,False,False,False,False,False,False,True,False,False,...,-358,-0.1,-330,-0.1,-16.299000,19.153999,-12.640000,AOC3_HUMAN,756,False
757,G,False,False,False,False,False,True,False,False,False,...,-360,-0.2,-1,-0.1,-13.404000,19.502001,-10.121000,AOC3_HUMAN,757,False
758,G,False,False,False,False,False,True,False,False,False,...,0,0.0,0,0.0,-10.986000,20.320000,-13.016000,AOC3_HUMAN,758,False


### Preprocess and analyze data

In [379]:
print(df_train.dtypes)

annotation_sequence     object
feat_A                    bool
feat_C                    bool
feat_D                    bool
feat_E                    bool
feat_F                    bool
feat_G                    bool
feat_H                    bool
feat_I                    bool
feat_K                    bool
feat_L                    bool
feat_M                    bool
feat_N                    bool
feat_P                    bool
feat_Q                    bool
feat_R                    bool
feat_S                    bool
feat_T                    bool
feat_V                    bool
feat_W                    bool
feat_Y                    bool
annotation_atomrec      object
feat_PHI               float64
feat_PSI               float64
feat_TAU               float64
feat_THETA             float64
feat_BBSASA            float64
feat_SCSASA            float64
feat_pLDDT             float64
feat_DSSP_H               bool
feat_DSSP_B               bool
feat_DSSP_E               bool
feat_DSS

In [380]:
df_train['y_Ligand'].value_counts()

False    479912
True      17254
Name: y_Ligand, dtype: int64

In [381]:
df_train['feat_PHI']

0      0.000000
1     -1.100680
2     -1.295398
3     -2.352796
4     -1.134474
         ...   
755   -2.378927
756   -2.122860
757   -1.124856
758    1.651085
759   -1.935096
Name: feat_PHI, Length: 497166, dtype: float64

In [382]:
df_train['entry']

0      GEMI5_HUMAN
1      GEMI5_HUMAN
2      GEMI5_HUMAN
3      GEMI5_HUMAN
4      GEMI5_HUMAN
          ...     
755     AOC3_HUMAN
756     AOC3_HUMAN
757     AOC3_HUMAN
758     AOC3_HUMAN
759     AOC3_HUMAN
Name: entry, Length: 497166, dtype: object

In [383]:
df_train['entry'].value_counts()

MACF1_HUMAN    7385
HUWE1_HUMAN    4371
DMD_HUMAN      3682
NF1_HUMAN      2836
SETD2_HUMAN    2561
               ... 
TVB9_HUMAN      111
CISD1_HUMAN     105
THIO_HUMAN      102
S1A7A_HUMAN      98
ACBD7_HUMAN      85
Name: entry, Length: 723, dtype: int64

## Transformation of Data

#### Group Column Information

In [384]:
df_aminoAcids = df_train[['feat_A', 'feat_C', 'feat_D', 'feat_E', 'feat_F',
       'feat_G', 'feat_H', 'feat_I', 'feat_K', 'feat_L', 'feat_M', 'feat_N',
       'feat_P', 'feat_Q', 'feat_R', 'feat_S', 'feat_T', 'feat_V', 'feat_W',
       'feat_Y']]
df_aminoAcids = df_aminoAcids * 1

df_proteinChainBonds = df_train[['feat_PHI', 'feat_PSI', 'feat_TAU', 'feat_THETA']]

df_y = df_train[['y_Ligand']]


In [385]:
# Add extra columns
frames = [df_aminoAcids, df_proteinChainBonds, df_y]
df_cleaned = pd.concat(frames, axis=1)
df_cleaned

,feat_A,feat_C,feat_D,feat_E,feat_F,feat_G,feat_H,feat_I,feat_K,feat_L,...,feat_S,feat_T,feat_V,feat_W,feat_Y,feat_PHI,feat_PSI,feat_TAU,feat_THETA,y_Ligand
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.000000,2.257610,-2.375020,1.956201,False
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,-1.100680,2.224168,-2.654037,1.900792,False
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,-1.295398,2.676551,-1.696727,2.458310,False
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,-2.352796,2.665542,-2.810012,2.054226,False
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,-1.134474,2.612150,-2.754863,2.272191,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,-2.378927,2.608671,-2.290233,2.192222,False
756,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,-2.122860,2.441583,-2.331874,1.570277,False
757,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,-1.124856,-0.248235,-1.292085,2.315429,False
758,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1.651085,-2.916255,-0.280431,2.332004,False


In [386]:
listOfAngles = ['feat_PHI', 'feat_PSI', 'feat_TAU', 'feat_THETA']
def convertColToDegree(col):
    df_cleaned[col] = np.rad2deg(df_cleaned[col])
    df_cleaned[col] %= 360

for val in listOfAngles:
    convertColToDegree(val)

df_cleaned

,feat_A,feat_C,feat_D,feat_E,feat_F,feat_G,feat_H,feat_I,feat_K,feat_L,...,feat_S,feat_T,feat_V,feat_W,feat_Y,feat_PHI,feat_PSI,feat_TAU,feat_THETA,y_Ligand
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.000000,129.351510,223.921383,112.082037,False
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,296.935674,127.435429,207.934896,108.907361,False
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,285.779139,153.355058,262.784699,140.850798,False
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,225.194735,152.724284,198.998174,117.698469,False
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,294.999412,149.665171,202.157972,130.186961,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,223.697523,149.465830,228.779305,125.605081,False
756,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,238.369087,139.892404,226.393461,89.970221,False
757,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,295.550499,345.777191,285.968994,132.664331,False
758,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,94.600182,192.910889,343.932504,133.613988,False


In [403]:
df_train['feat_BBSASA']

0      80.020602
1      69.542382
2      23.387401
3       4.908812
4       9.742674
         ...    
755     7.313673
756    20.300374
757     2.763823
758    15.092203
759    20.331329
Name: feat_BBSASA, Length: 497166, dtype: float64

#### Make Undersampled Dataset

In [387]:
# Split Data
X = df_cleaned.iloc[:,0:len(df_cleaned.columns)-1]
Y = df_cleaned.iloc[:,-1]
X

,feat_A,feat_C,feat_D,feat_E,feat_F,feat_G,feat_H,feat_I,feat_K,feat_L,...,feat_R,feat_S,feat_T,feat_V,feat_W,feat_Y,feat_PHI,feat_PSI,feat_TAU,feat_THETA
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000000,129.351510,223.921383,112.082037
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,296.935674,127.435429,207.934896,108.907361
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,285.779139,153.355058,262.784699,140.850798
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,225.194735,152.724284,198.998174,117.698469
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,294.999412,149.665171,202.157972,130.186961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,223.697523,149.465830,228.779305,125.605081
756,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,238.369087,139.892404,226.393461,89.970221
757,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,295.550499,345.777191,285.968994,132.664331
758,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,94.600182,192.910889,343.932504,133.613988


In [388]:
Y

0      False
1      False
2      False
3      False
4      False
       ...  
755    False
756    False
757    False
758    False
759    False
Name: y_Ligand, Length: 497166, dtype: bool

In [389]:
# Undersample, apply random sampler
print(Counter(Y))

undersample = RandomUnderSampler(sampling_strategy='majority')
X_US, Y_US = undersample.fit_resample(X, Y)
print(Counter(Y_US))

Counter({False: 479912, True: 17254})
Counter({False: 17254, True: 17254})


## What Factors predict drug binding or non-drug binding?

#### Hypothesis Testing (Univariate Selection with ChiSquared Test)

In [390]:
# apply the SelectKBest class to extract the top features 
def SelectKBestFunc(X, Y):
    """
    SelectKBestFunc takes in feature data, and target data, returns a fitted chi2 model between features and target
    """
    bestFeatures = SelectKBest(score_func=chi2, k='all')
    fit = bestFeatures.fit(X, Y)
    return fit


def printSummary(type, fit):
    """
    printSummary takes in a type of dataset, and a fitted model, and prints the 10 best features
    """
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(X.columns)
    dfpvalues = pd.DataFrame(fit.pvalues_)

    featureScores = pd.concat([dfcolumns, dfscores, dfpvalues], axis=1)
    featureScores.columns = ['Specs', 'Score', 'p-value']

    print(type)
    print(featureScores.nlargest(10, 'Score'))

In [391]:
featureScoreUndersampled = SelectKBestFunc(X_US, Y_US)
printSummary("Undersample Dataset", featureScoreUndersampled)

Undersample Dataset
       Specs        Score        p-value
20  feat_PHI  3072.515581   0.000000e+00
22  feat_TAU  2412.246472   0.000000e+00
21  feat_PSI   575.433605  3.692933e-127
12    feat_P   208.557789   2.834787e-47
5     feat_G   150.503226   1.345760e-34
19    feat_Y   109.424460   1.310030e-25
4     feat_F   106.441999   5.899412e-25
3     feat_E    68.896350   1.037767e-16
13    feat_Q    67.993620   1.640254e-16
18    feat_W    45.062157   1.908786e-11


#### Recursive Feature Elimination using RandomForestClassifier, DecisionTreeClassifier

In [392]:
def RFEFunc(X_one, Y_one, model):
    rfe = RFE(estimator=model, n_features_to_select=5)
    fit = rfe.fit(X_one, Y_one)
    arrFinal = []

    print("Num Features:", fit.n_features_)
    print("Best Selected Features:")
    arrBool = fit.support_
    for i in range(0, len(arrBool)):
        if(arrBool[i] == True):
            arrFinal.append(X.columns[i])

    print(*arrFinal, sep='\n')
    return arrFinal

In [393]:
arrUSLR = RFEFunc(X_US, Y_US, LogisticRegression())
arrUSRFC = RFEFunc(X_US, Y_US, RandomForestClassifier())
arrUSXGB = RFEFunc(X_US, Y_US, xgboost.XGBClassifier())

Num Features: 5
Best Selected Features:
feat_F
feat_G
feat_P
feat_W
feat_Y
Num Features: 5
Best Selected Features:
feat_P
feat_PHI
feat_PSI
feat_TAU
feat_THETA
Num Features: 5
Best Selected Features:
feat_E
feat_F
feat_G
feat_P
feat_Y


In [394]:
X_USLR = X_US[arrUSLR]
X_USRFC = X_US[arrUSRFC]
X_USXGB = X_US[arrUSXGB]

In [395]:
x_train_uslr, x_test_uslr, y_train_uslr, y_test_uslr = train_test_split(X_USLR, Y_US, test_size = 0.2, random_state=42)
x_train_usrfc, x_test_usrfc, y_train_usrfc, y_test_usrfc = train_test_split(X_USRFC, Y_US, test_size = 0.2, random_state=42)
x_train_usxgb, x_test_usxgb, y_train_usxgb, y_test_usxgb = train_test_split(X_USXGB, Y_US, test_size = 0.2, random_state=42)

In [396]:
y_test_usrfc.value_counts()

False    3466
True     3436
Name: y_Ligand, dtype: int64

#### Run Logistic Regression Model

In [397]:
lr = LogisticRegression()
lr.fit(x_train_uslr, y_train_uslr)
predictionsOne = lr.predict(x_test_uslr)
lr.score(x_test_uslr, y_test_uslr)

classificationReportOne = metrics.classification_report(y_test_uslr, predictionsOne)
confusionMatrixOne = metrics.confusion_matrix(y_test_uslr, predictionsOne)
print("Classification Report on Undersampled Dataset")
print(classificationReportOne)
print(confusionMatrixOne)
print('F1 Score: %.3f' % metrics.f1_score(y_test_uslr, predictionsOne))

Classification Report on Undersampled Dataset
              precision    recall  f1-score   support

       False       0.53      0.86      0.66      3466
        True       0.62      0.23      0.33      3436

    accuracy                           0.55      6902
   macro avg       0.57      0.54      0.49      6902
weighted avg       0.57      0.55      0.50      6902

[[2978  488]
 [2649  787]]
F1 Score: 0.334


In [398]:
fpr, tpr, thresholds = metrics.roc_curve(y_test_uslr, predictionsOne, pos_label=1)
auc_roc = metrics.auc(fpr, tpr)

precision, recall, _ = metrics.precision_recall_curve(y_test_uslr, predictionsOne)
auc_pr = metrics.auc(recall, precision)

print(f"ROC-AUC: {auc_roc} \n PR-AUC {auc_pr}")

ROC-AUC: 0.544124547323845 
 PR-AUC 0.6150510500856439


#### Run Random Forest Model

In [399]:
randomForestUS = RandomForestClassifier()
randomForestUS.fit(x_train_usrfc, y_train_usrfc)
predictionsTwo = randomForestUS.predict(x_test_usrfc)
randomForestUS.score(x_test_usrfc, y_test_usrfc)

classificationReportTwo = metrics.classification_report(y_test_usrfc, predictionsTwo)
confusionMatrixTwo = metrics.confusion_matrix(y_test_usrfc, predictionsTwo)
print("Classification Report on Undersampled Dataset")
print(classificationReportTwo)
print(confusionMatrixTwo)
print('F1 Score: %.3f' % metrics.f1_score(y_test_usrfc, predictionsTwo))

Classification Report on Undersampled Dataset
              precision    recall  f1-score   support

       False       0.63      0.62      0.63      3466
        True       0.62      0.63      0.63      3436

    accuracy                           0.63      6902
   macro avg       0.63      0.63      0.63      6902
weighted avg       0.63      0.63      0.63      6902

[[2159 1307]
 [1277 2159]]
F1 Score: 0.626


In [400]:
fpr, tpr, thresholds = metrics.roc_curve(y_test_usrfc, predictionsTwo, pos_label=1)
auc_roc = metrics.auc(fpr, tpr)

precision, recall, _ = metrics.precision_recall_curve(y_test_usrfc, predictionsTwo)
auc_pr = metrics.auc(recall, precision)

print(f"ROC-AUC: {auc_roc} \n PR-AUC {auc_pr}")

ROC-AUC: 0.625627583302153 
 PR-AUC 0.7181370008622805


#### Run XGBoost Model

In [401]:
xgb = xgboost.XGBClassifier()
xgb.fit(x_train_usxgb, y_train_usxgb)
predictionsThree = xgb.predict(x_test_usxgb)
xgb.score(x_test_usxgb, y_test_usxgb)

classificationReportThree = metrics.classification_report(y_test_usxgb, predictionsThree)
confusionMatrixThree = metrics.confusion_matrix(y_test_usxgb, predictionsThree)
print("Classification Report on Undersampled Dataset")
print(classificationReportThree)
print(confusionMatrixThree)
print('F1 Score: %.3f' % metrics.f1_score(y_test_usxgb, predictionsThree))

Classification Report on Undersampled Dataset
              precision    recall  f1-score   support

       False       0.53      0.87      0.66      3466
        True       0.61      0.21      0.31      3436

    accuracy                           0.54      6902
   macro avg       0.57      0.54      0.48      6902
weighted avg       0.57      0.54      0.48      6902

[[3014  452]
 [2723  713]]
F1 Score: 0.310


In [402]:
fpr, tpr, thresholds = metrics.roc_curve(y_test_usxgb, predictionsThree, pos_label=1)
auc_roc = metrics.auc(fpr, tpr)

precision, recall, _ = metrics.precision_recall_curve(y_test_usxgb, predictionsThree)
auc_pr = metrics.auc(recall, precision)

print(f"ROC-AUC: {auc_roc} \n PR-AUC {auc_pr}")

ROC-AUC: 0.538549518455349 
 PR-AUC 0.6070246125183183
